In [1]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    DiscreteContactApproximation,
    InverseKinematics,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    RigidTransform,
    RotationMatrix,
    Solve,
    StartMeshcat,
    WeldJoint,
    eq,
)

from manipulation import ConfigureParser, running_as_notebook

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://62354575-ad81-482c-adda-c71e1a9422a9.deepnoteproject.com/7000/


## Problem Description
We will solve inverse kinematics for one arm on a PR2 robot. This robot model has been modified to have convex collision geometries, and to simplify this problem, we have welded (fixed) several joints that are irrelavent to the kinematics of the arms.

In [3]:
# TODO(russt): Move this to Drake.


def ReplaceJointWithWeld(plant, joint):
    for actuator in [
        plant.get_joint_actuator(index) for index in plant.GetJointActuatorIndices()
    ]:
        if actuator.joint() == joint:
            plant.RemoveJointActuator(actuator)

    weld = WeldJoint(
        joint.name(), joint.frame_on_parent(), joint.frame_on_child(), RigidTransform()
    )
    plant.RemoveJoint(joint)
    plant.AddJoint(weld)


def build_env():
    """Load in models and build the diagram."""
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.01)
    plant.set_discrete_contact_approximation(DiscreteContactApproximation.kSap)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://manipulation/pr2_shelves.dmd.yaml")
    # Remove mimic constraints:
    for id in plant.GetConstraintIds():
        plant.RemoveConstraint(id)
    # Remove some superfluous joints
    ReplaceJointWithWeld(plant, plant.GetJointByName("head_pan_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("head_tilt_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_l_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_r_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_l_finger_tip_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("r_gripper_r_finger_tip_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_l_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_r_finger_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_l_finger_tip_joint"))
    ReplaceJointWithWeld(plant, plant.GetJointByName("l_gripper_r_finger_tip_joint"))
    plant.Finalize()

    MeshcatVisualizer.AddToBuilder(
        builder,
        scene_graph.get_query_output_port(),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    diagram = builder.Build()
    return diagram, plant, scene_graph

In [4]:
goal_rotation = RotationMatrix(
    [
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1],
    ]
)
goal_position = np.array([-0.83, 0.18, 1.4])
goal_pose = RigidTransform(goal_rotation, goal_position)

### IK for a Mobile Manipulator

Given a RigidTransform X_WG, compute a robot configuration placing the left gripper at that pose. We use optimization to solve the IK problem, and we repeatedly solve the program with random initializations until it succeeds. We have implemented a skeleton of the necessary code in the following function, but you must complete several pieces:

- Add position and orientation constraints to the gripper frame. The end effector should match the desired pose to within 1mm translation along each axis, and the rotation should be off by no more than 1 degree.
- If `fix_base` is True, constrain the base pose $(x,y,\theta)$ to be equal to `base_pose`.
- Add a collision free constraint with [AddMinimumDistanceLowerBoundConstraint](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_inverse_kinematics.html#a2ecd71efd675a7e1a4293adb05c9b9df). The minimum distance between any pair of collision geometries should be at least 1cm.
- Compute a random initial guess for the joint angles within the robot's joint limits. You can access the joint limits from the multibody plant, but some of the joints are angle-valued and don't have limits. For these joints, use the range $[-\pi,\pi]$.

In [5]:
def solve_ik(X_WG, max_tries=10, fix_base=False, base_pose=np.zeros(3)):
    diagram, plant, scene_graph = build_env()

    plant.GetFrameByName("l_gripper_palm_link")

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    scene_graph.GetMyContextFromRoot(context)

    # Note: passing in a plant_context is necessary for collision-free constraints!
    ik = InverseKinematics(plant, plant_context)
    q_variables = ik.q()  # Get variables for MathematicalProgram
    prog = ik.prog()  # Get MathematicalProgram
    q_nominal = np.zeros(len(q_variables))
    q_nominal[0:3] = base_pose
    prog.AddQuadraticErrorCost(np.eye(len(q_variables)), q_nominal, q_variables)

    # Add your constraints here
    p_WG = X_WG.translation()
    R_WG = X_WG.rotation()

    p_WG_lower = p_WG - 0.001
    p_WG_upper = p_WG + 0.001
    bounds = np.deg2rad(1)

    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("l_gripper_palm_link")
    ik.AddPositionConstraint(
        frameA=world_frame,
        frameB=gripper_frame,
        p_BQ=np.zeros(3),
        p_AQ_lower=p_WG_lower,
        p_AQ_upper=p_WG_upper,
        )

    ik.AddOrientationConstraint(
        frameAbar=world_frame,
        R_AbarA=R_WG,
        frameBbar=gripper_frame,
        R_BbarB=RotationMatrix(),
        theta_bound=bounds,
        )

    # If fix_base is True, constrain the base pose 
    if fix_base == True:
        prog.AddConstraint(eq(q_variables[0:3], base_pose))
    
    ik.AddMinimumDistanceLowerBoundConstraint(0.01)

    for count in range(max_tries):
        # Compute a random initial guess here
        rng = np.random.default_rng()
        prog.SetInitialGuess(q_variables,rng.uniform(-np.pi, np.pi, len(q_variables)))
    
        result = Solve(prog)

        if running_as_notebook:
            render_context = diagram.CreateDefaultContext()
            plant.SetPositions(
                plant.GetMyContextFromRoot(render_context),
                result.GetSolution(q_variables),
            )
            diagram.ForcedPublish(context)

        if result.is_success():
            print("Succeeded in %d tries!" % (count + 1))
            return result.GetSolution(q_variables)

    print("Failed!")
    return None

First, we show an example where we have fixed the base. It may take many tries for it to solve, or not solve at all! At each iteration, we visualize where the optimizer stopped, so you can see what the failures look like.

In [6]:
solve_ik(
    goal_pose,
    max_tries=20,
    fix_base=True,
    base_pose=np.array([-1.23, 0.05, 0]),
)

Succeeded in 2 tries!


array([-1.23000000e+00,  5.00000000e-02,  0.00000000e+00,  9.40168054e-02,
       -2.10149849e-01,  1.74424160e-01, -7.11506503e-02, -3.71200908e-05,
        6.95987856e-05,  0.00000000e+00,  2.35749628e-04, -2.69444409e-01,
       -5.23600000e-01, -2.36252033e-01, -1.04835817e+00, -9.02902761e+00,
       -1.50387942e+00, -6.07992797e+00])

When we allow the base to move freely, the inverse kinematics can be solved much more easily!

In [7]:
solve_ik(goal_pose, fix_base=False)

Succeeded in 1 tries!


array([-1.52919671e+00,  3.43087613e-01, -4.29064329e-01,  3.10000000e-01,
       -5.52657163e-02, -4.38496832e-02, -9.29518184e-03,  0.00000000e+00,
        5.60297394e-06, -1.58444036e-05,  5.99727449e-06, -6.07444581e-02,
       -4.26586476e-01, -8.00000000e-01,  0.00000000e+00, -1.40792997e+00,
       -6.31116819e-01, -8.12403528e-01])

In [8]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.mobile.test_mobile_base_ik import TestMobileBaseIk

Grader.grade_output([TestMobileBaseIk], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 5/5.

Score for Test solve_ik is 5/5.
- Succeeded in 3 tries!
Succeeded in 2 tries!
Succeeded in 3 tries!
Failed!



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=62354575-ad81-482c-adda-c71e1a9422a9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>